In [36]:
from scimath.units.energy import eV, KeV
from scimath.units.api import UnitScalar
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import random

### Rutherford

#### 1) Theoretical

In [7]:
import numpy as np

# angular differential cross section
#@has_units
def alpha(energy, Z):
    ''' alpha parameter used in Rutherford scattering

        Parameters
        ----------
        energy : array : units = eV

        Z      : array : units = dim

        Returns
        -------
        alpha  : array : units = dim
    '''

    return  3.4*(Z**(2/3)) / energy

#@has_units
def sigma(E, Z):
    """ Calculate the relativistic Rutherford elastic cross section
        per atom
        
        Parameters
        ----------
        E      : array : units = eV
        
        Z      : array : units = dim        
        
        Returns
        -------
        s_R    : array : units = cm**2       
    """      
    alphaR = alpha(E, Z)

    E = E * eV/KeV
    
    s_R = 5.21e-21 * Z*Z * (4.*np.pi)/(alphaR*(1. + alphaR)) * ((E + 511.)/(E + 1024.)/E)**2
    return s_R

#@has_units
def dSigma_dTheta(E, Z, angle):
    """ Calculate the angular differential cross section
        
        Parameters
        ----------
        E      : array : units = eV
        
        Z      : array : units = dim        
        
        angle  : array : units = dim
        
        Returns
        -------
        dS_dTheta   : array : units = dim      
    """     
    alphaR = alpha(E, Z)
    
    E = E * eV/KeV
    maxVal =  5.21e-21 * Z*Z * ((E+511)/(E+1024)/E)**2 / (alphaR**2)
    dS_dTheta = 5.21e-21 * Z*Z * ((E+511)/(E+1024)/E)**2 /(((np.sin(np.radians(angle*0.5)))**2 + alphaR)**2)

    return dS_dTheta /maxVal


def theta(E, Z, rands):
    ''' Rutherford scattering angle has a simple
    form when using dOmega. 

    See Joy pg 30
    '''
    alphaR = alpha(E,Z)
    
    return np.arccos(1 - 2*alphaR*rands/(1+alphaR-rands) )


### Scattering variables for Al

In [29]:
Z_Al = 13
# incident energy
E_Al = UnitScalar(20000, units="eV")

#### Rutherford scattering angle probability

In [38]:
angles = np.linspace(0, 180, 180)

t_analytical = go.Scatter(
    x = angles,
    y = np.array(dSigma_dTheta(E_Al, Z_Al, angles)),
    mode = 'lines', 
    name = 'analytical', 
)

layout = go.Layout(
    title = 'Rutherford', 

    xaxis=dict(
        title = 'Scattering angle (degrees)'
    ), 
    yaxis=dict(
        type='log',
        title = 'Distribution per scattering event'
    )
)


data = [t_analytical]
fig = go.Figure(data=data, layout=layout)

fig.show()

#### Compare this with using equation (1) from Casino[...] part II

In [39]:
rands = np.random.rand(10000000)

histy, histx = np.histogram(np.degrees(theta(E_Al, Z_Al, rands)), bins = 180, range = (0, 180))

t_sample = go.Scatter(
    x = (np.array(histx[:-1]) + np.array(histx[1:]))/2,
    y = histy/max(histy),
)


layout = go.Layout(
    title = 'Rutherford', 

    xaxis=dict(
        title = 'Scattering angle (degrees)'
    ), 
    yaxis=dict(
        type='log',
        title = 'Distribution per scattering event'
    )
)

data = [t_sample]
fig = go.Figure(data=data, layout=layout)

fig.show()

In [32]:
from scipy.integrate import quad

def integrand (theta, E, Z):
    ''' F(theta) dTheta = dSigma/dTheta/Sigma dTheta  
        
        F(theta) = probability distribution of scattering angle being < theta
    '''
    return np.array(dSigma_dTheta(E, Z, theta)/sigma(E, Z)) 
    
    
def funcTheta(theta, E, Z):
    ''' normalised integral F(theta)dTheta = [0..1]
    
    theta = degrees
    '''
    intTheta, _ = quad(integrand, 0, theta, args=(E,Z))
    intTot, _ = quad(integrand, 0, 180, args=(E,Z))
    return intTheta/intTot


def funcOmega(theta, E, Z):
    '''
    normalised intergral F(omega)dOmega = [0..1]
    
    This form has an easier analytical result
    '''
    return (np.cos(np.radians(theta))-1)*(1+alpha(E, Z,))/(np.cos(np.radians(theta))-1-2*alpha(E, Z,))


In [40]:
angles = np.linspace(0, 180, 360)

prob = []
for angle in angles:
    prob.append(funcTheta(angle, float(E_Al), Z_Al))


t_theta = go.Scatter(
    x = angles,
    y = np.array(prob),
    mode = 'lines', 
    name = 'dTheta', 
)

t_omega = go.Scatter(
    x = angles,
    y = (funcOmega(angles, float(E_Al), Z_Al)),
    mode = 'lines', 
    name = 'dOmega', 
)
layout = go.Layout(
    title = 'The random number samples from these distributions', 

    xaxis=dict(
        title = 'Scattering angle (degrees)'
    ), 
    yaxis=dict(
        title = 'uniformly distributed scattering angle (R)'
    )
)

data = [t_theta, t_omega]
fig = go.Figure(data=data, layout=layout)

fig.show()

In [34]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'polar'}]*1]*1)


fig.add_trace( go.Scatterpolar(
        name = 'f(Theta)',
        r = np.array(dSigma_dTheta(E_Al, Z_Al, angles)),
        theta = angles,
        mode = 'lines',
    ), 1, 1)

fig.add_trace( go.Scatterpolar(
        name = 'f(Omega)', 
        r = histy/max(histy),
        theta = histx[:-1],
        mode = 'lines',
    ), 1, 1)

fig.update_layout(
    title = 'Polar angular distributions',
    polar =dict(
      radialaxis = dict(type = "log", tickangle = 45),
      sector = [0, 180]
    ), 
)

fig.show()